# Using BERTopic

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
seed = 2022

In [2]:
csv_file = "cleaned_EThOS_CSV_202210.csv"
df = pd.read_csv(csv_file, encoding="ISO-8859-2")
df.head()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_12874/740333908.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, encoding="ISO-8859-2")


,Unnamed: 0,Title,DOI,Author,Author ISNI,ORCID,Institution,Institution ISNI,Date,Qualification,Abstract,Subject Discipline,Supervisor(s),Funder(s),EThOS URL,IR URL
0,0,Computation and measurement of turbulent flow ...,,"Loizou, Panos A.",https://isni.org/isni/0000000136122593,,University of Manchester,https://isni.org/isni/0000000121662407,1989,Thesis (Ph.D.),,Physical Sciences,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,
1,1,Prolactin and growth hormone secretion in norm...,,"Prescott, R. W. G.",https://isni.org/isni/0000000134992122,,University of Newcastle upon Tyne,https://isni.org/isni/0000000104627212,1983,Thesis (Ph.D.),,Biological Sciences,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,
2,2,Influence of strain fields on flame propagation,,"Mendes-Lopes, J. M. C.",https://isni.org/isni/0000000133937105,,University of Cambridge,https://isni.org/isni/0000000121885934,1983,Thesis (Ph.D.),Flame propagation can be highly influenced by ...,Engineering & Technology,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,
3,3,"Connectivity, flow and transport in network mo...",,"Robinson, Peter Clive",https://isni.org/isni/0000000135292822,,University of Oxford,https://isni.org/isni/0000000419368948,1984,Thesis (Ph.D.),In order to evaluate the safety of radioactive...,Physical Sciences,"Rae, John",,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,http://ora.ox.ac.uk/objects/uuid:c61bf73b-b794...
4,4,The theory and implementation of a high qualit...,,"Lower, K. N.",https://isni.org/isni/0000000136133567,,University of Bristol,https://isni.org/isni/0000000419367603,1985,Thesis (Ph.D.),,Engineering & Technology,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,


## Create new column concatenating Title and Abstract

In [3]:
df["Text"] = df["Title"] + ' : ' + df["Abstract"]

In [4]:
df["Title"].iloc[2]

'Influence of strain fields on flame propagation'

In [5]:
df["Abstract"].iloc[2]

'Flame propagation can be highly influenced by the presence of strain fields. Two regimes of turbulent flame propagation can be identified: (a) a strain-dominated regime which occurs when the smallest eddies are larger than the laminar flame thickness; and (b) a mixing-dominated regime found when the smallest eddies are smaller than the laminar flame thickness. Therefore, flame propagation in a low to moderate intensity turbulent medium, and initial stages of flame growth from a point ignition source, may be dominated by straining effects. This is because in these cases it is very likely that the laminar flame thickness is smaller than the Kolmogorov length scale, which is a measure of the smallest structures of the turbulence. In this dissertation theoretical and experimental work is reported on the influence of a uniform strain field (together with heat loss) on laminar flame propagation. The theoretical results show that, in general, the laminar burning velocity decreases when the s

In [6]:
df["Text"].iloc[2]

'Influence of strain fields on flame propagation : Flame propagation can be highly influenced by the presence of strain fields. Two regimes of turbulent flame propagation can be identified: (a) a strain-dominated regime which occurs when the smallest eddies are larger than the laminar flame thickness; and (b) a mixing-dominated regime found when the smallest eddies are smaller than the laminar flame thickness. Therefore, flame propagation in a low to moderate intensity turbulent medium, and initial stages of flame growth from a point ignition source, may be dominated by straining effects. This is because in these cases it is very likely that the laminar flame thickness is smaller than the Kolmogorov length scale, which is a measure of the smallest structures of the turbulence. In this dissertation theoretical and experimental work is reported on the influence of a uniform strain field (together with heat loss) on laminar flame propagation. The theoretical results show that, in general,

Writing a function that obtains the dataframe for a particular subject:

In [31]:
def obtain_subject_df(df, subject):
    subject_df = df[df["Subject Discipline"]==subject].reset_index()
    subject_df["Date"] = subject_df["Date"].astype(int)
    return subject_df

## Computer Science topic modelling

Let's only consider computer science for now:

In [38]:
cs_df = obtain_subject_df(df, "Computer Science")

In [41]:
cs_df.head()

,index,Unnamed: 0,Title,DOI,Author,Author ISNI,ORCID,Institution,Institution ISNI,Date,Qualification,Abstract,Subject Discipline,Supervisor(s),Funder(s),EThOS URL,IR URL,Text
0,13,13,A methodology for automated design of computer...,,"Bennett, J. P.",https://isni.org/isni/0000000134590959,,University of Cambridge,https://isni.org/isni/0000000121885934,1987,Thesis (Ph.D.),,Computer Science,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,A methodology for automated design of computer...
1,14,14,Reasoning about the function and timing of int...,,"Leeser, Miriam Ellen",https://isni.org/isni/0000000136076953,,University of Cambridge,https://isni.org/isni/0000000121885934,1987,Thesis (Ph.D.),The structure of circuits is specified with Pr...,Computer Science,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,Reasoning about the function and timing of int...
2,57,57,Learning to classify from temporal data in the...,,"Black, Michaela",,,Ulster University,https://isni.org/isni/0000000105519715,2002,Thesis (Ph.D.),,Computer Science,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,Learning to classify from temporal data in the...
3,71,71,Rational agents and the processes and states o...,,"Paurobally, Shamimabi",,,Imperial College London,https://isni.org/isni/0000000121889740,2002,Thesis (Ph.D.),,Computer Science,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,Rational agents and the processes and states o...
4,78,78,Optical packet networking using optical time d...,,"Gao, Ruixin",https://isni.org/isni/0000000134879699,,Sheffield Hallam University,https://isni.org/isni/000000010303540X,2003,Thesis (Ph.D.),Growing demands for capacity have stimulated t...,Computer Science,"Ghassemlooy, Z. ; Ball, Peter ; Swift, Graham",,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,http://shura.shu.ac.uk/19219/,Optical packet networking using optical time d...


In [34]:
len(cs_df)

14401

## Using BERTopic

In [11]:
from bertopic import BERTopic

### Modelling choices

Choosing the representation model for BERTopic (i.e. how we choose the final set of words that describe a collection of documents):

In [12]:
rep_model_choice = "keybert"

if rep_model_choice == "openai":
    import openai
    from bertopic.representation import OpenAI
    openai.api_type = 'azure'
    openai.api_key = os.environ["OPENAI_API_KEY"]
    representation_model = OpenAI(model="gpt-3.5-turbo", chat=True)
elif rep_model_choice == "hf":
    from bertopic.representation import TextGeneration
    representation_model = TextGeneration("gpt2")
elif rep_model_choice == "keybert":
    from bertopic.representation import KeyBERTInspired
    representation_model = KeyBERTInspired()
else:
    raise ValueError
    
representation_model

KeyBERTInspired()

Choosing the dimensionality reduction model:

In [42]:
# logical value to determine whether or not to dimension reduce (using UMAP)
skip_umap = False

if skip_umap:
    from bertopic.dimensionality import BaseDimensionalityReduction
    umap_model = BaseDimensionalityReduction()
else:
    from umap import UMAP
    umap_model = UMAP(n_neighbors=15,
                      n_components=5,
                      min_dist=0.0,
                      metric='cosine',
                      random_state=seed)
    
umap_model

UMAP(metric='cosine', min_dist=0.0, n_components=5, random_state=2022)

Choosing the sentence-transformer to use:

In [14]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L12-v2")

### Creating and fitting model

In [15]:
# create and fit topic model
topic_model = BERTopic(min_topic_size=40,
                       embedding_model=sentence_model,
                       umap_model=umap_model,
                       representation_model=representation_model)
topics, probs = topic_model.fit_transform(cs_df["Text"])

In [16]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5398,-1_systems_computing_software_data,"[systems, computing, software, data, networks,...",[The Sea of Stuff : a model to manage shared m...
1,0,829,0_parallel_parallelism_concurrency_processors,"[parallel, parallelism, concurrency, processor...",[Engineering the performance of parallel appli...
2,1,609,1_vr_usability_virtual_interactive,"[vr, usability, virtual, interactive, design, ...",[Augmented analyses : supporting the study of ...
3,2,553,2_logics_specification_logic_reasoning,"[logics, specification, logic, reasoning, comp...",[Logic programming tools and techniques for im...
4,3,498,3_recognition_vision_images_segmentation,"[recognition, vision, images, segmentation, de...",[Multi evidence fusion scheme for content-base...
5,4,437,4_nlp_linguistic_lexical_corpus,"[nlp, linguistic, lexical, corpus, sentiment, ...",[Unsupervised and knowledge-poor approaches to...
6,5,421,5_agents_agent_autonomous_robots,"[agents, agent, autonomous, robots, reinforcem...",[Knowledge-based reward shaping with knowledge...
7,6,397,6_software_application_developers_specification,"[software, application, developers, specificat...",[Quality metrics in software engineering : In ...
8,7,391,7_classifiers_classifier_classification_superv...,"[classifiers, classifier, classification, supe...",[Flexible and efficient Gaussian process model...
9,8,373,8_heuristics_optimization_heuristic_optimisation,"[heuristics, optimization, heuristic, optimisa...",[A genetic algorithm approach for combinatoria...


Let's look a bit more into these words for each of the main topics

In [17]:
topic_model.get_topic_info(0)["Representation"][0]

['parallel',
 'parallelism',
 'concurrency',
 'processors',
 'concurrent',
 'programming',
 'processor',
 'program',
 'computing',
 'cores']

In [18]:
topic_model.get_topic_info(1)["Representation"][0]

['vr',
 'usability',
 'virtual',
 'interactive',
 'design',
 'technologies',
 'interaction',
 'technology',
 'environments',
 'computer']

In [19]:
topic_model.get_topic_info(2)["Representation"][0]

['logics',
 'specification',
 'logic',
 'reasoning',
 'compiler',
 'implementation',
 'semantics',
 'programming',
 'program',
 'theories']

## Looking at topic development over time

In [43]:
topics_over_time = topic_model.topics_over_time(docs=cs_df["Text"],
                                                timestamps=cs_df["Date"])

In [48]:
len(topics_over_time["Topic"].value_counts())

51

In [49]:
import math
topics_over_time["Timestamp"] = topics_over_time["Timestamp"].apply(lambda x: math.floor(x))

In [50]:
topics_over_time["Timestamp"].value_counts()

2013    51
2015    50
2010    49
2017    49
2014    49
        ..
1960     1
1953     1
1962     1
1964     1
1948     1
Name: Timestamp, Length: 64, dtype: int64

In [54]:
topics_over_time[topics_over_time["Timestamp"]==2013].sort_values("Frequency", ascending=False)

,Topic,Words,Frequency,Timestamp
1126,-1,"modelling, software, systems, models, model",226,2013
1128,1,"usability, interactive, interaction, vr, haptic",31,2013
1127,0,"concurrency, concurrent, processors, processor...",28,2013
1130,3,"recognition, vision, descriptors, images, dete...",24,2013
1129,2,"datatypes, types, semantics, type, logics",22,2013
1131,4,"nlp, corpus, lexicon, lexical, parsing",22,2013
1138,11,"neurons, neural, neuron, neuronal, synapses",22,2013
1137,10,"speech, voices, audio, acoustic, speaker",19,2013
1133,6,"models, modeling, model, specification, software",18,2013
1134,7,"classifiers, classifier, algorithms, classific...",16,2013


In [60]:
topic_model.get_topic_info(-1)["Representation"][0]

['systems',
 'computing',
 'software',
 'data',
 'networks',
 'cloud',
 'system',
 'models',
 'network',
 'modelling']

In [61]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

Okay, looks like we're on an absolute timeline here. Could we reproduce this so that it's a proportion too? Then we could see what were the dominating topics over time...

# Mathematics

In [62]:
mathematics_df = obtain_subject_df(df, "Mathematics & Statistics")

In [63]:
mathematics_df.head()

,index,Unnamed: 0,Title,DOI,Author,Author ISNI,ORCID,Institution,Institution ISNI,Date,Qualification,Abstract,Subject Discipline,Supervisor(s),Funder(s),EThOS URL,IR URL,Text
0,6,6,A unified approach to the identification of dy...,,"Brown, T. A.",https://isni.org/isni/0000000135047894,,University of Bristol,https://isni.org/isni/0000000419367603,1985,Thesis (Ph.D.),,Mathematics & Statistics,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,A unified approach to the identification of dy...
1,18,18,"On-line fault detection, a system-nonspecific ...",,"McMichael, D. W.",https://isni.org/isni/000000013625852X,,University of Oxford,https://isni.org/isni/0000000419368948,1987,Thesis (Ph.D.),,Mathematics & Statistics,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,"On-line fault detection, a system-nonspecific ..."
2,138,138,A dual-based approach to a multi-objective loc...,,"Abdel-Lateef, Bahgat Abdel-Hameed",https://isni.org/isni/0000000133894037,,University of Liverpool,https://isni.org/isni/0000000419368470,1988,Thesis (Ph.D.),,Mathematics & Statistics,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,A dual-based approach to a multi-objective loc...
3,187,187,Modules over group algebras which are free on ...,,"Andrews, Robert Charles",https://isni.org/isni/0000000134242794,,University of Warwick,https://isni.org/isni/0000000088091613,1987,Thesis (Ph.D.),Consider the following situation: k will be an...,Mathematics & Statistics,,Science and Engineering Research Council,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,http://wrap.warwick.ac.uk/98268/,Modules over group algebras which are free on ...
4,235,235,Simplicial decompositions and universal graphs,,"Diestal, R.",https://isni.org/isni/0000000134242612,,University of Cambridge,https://isni.org/isni/0000000121885934,1986,Thesis (Ph.D.),,Mathematics & Statistics,,,https://ethos.bl.uk/OrderDetails.do?uin=uk.bl....,,Simplicial decompositions and universal graphs...


In [64]:
len(mathematics_df)

12692

In [70]:
# create and fit topic model
topic_model_mathematics = BERTopic(min_topic_size=50,
                                   embedding_model=sentence_model,
                                   umap_model=umap_model,
                                   representation_model=representation_model)
topics, probs = topic_model_mathematics.fit_transform(mathematics_df["Text"])

In [71]:
topic_model_mathematics.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6062,-1_modelling_numerical_models_methods,"[modelling, numerical, models, methods, data, ...",[On the numerical solution of the filtering pr...
1,0,728,0_groups_subgroups_subgroup_group,"[groups, subgroups, subgroup, group, abelian, ...",[Algebraic modules for finite groups : The mai...
2,1,569,1_pdes_numerical_boundary_partial,"[pdes, numerical, boundary, partial, method, d...",[The boundary element method for the numerical...
3,2,311,2_chromatic_colourings_graphs_colouring,"[chromatic, colourings, graphs, colouring, gra...",[Colourings of random graphs : We study graph ...
4,3,303,3_logics_logic_axioms_specification,"[logics, logic, axioms, specification, reasoni...",[Abstraction barriers and refinement in the po...
5,4,279,4_flows_viscous_turbulence_flow,"[flows, viscous, turbulence, flow, turbulent, ...",[Leading edge instability and generation of To...
6,5,276,5_stochastic_diffusion_brownian_noise,"[stochastic, diffusion, brownian, noise, sdes,...",[On asymptotic stability of stochastic differe...
7,6,264,6_nonparametric_estimators_estimation_statistical,"[nonparametric, estimators, estimation, statis...",[Inference for generalised linear mixed models...
8,7,229,7_supersymmetric_monopoles_gauge_branes,"[supersymmetric, monopoles, gauge, branes, mon...",[Counting gauge invariant operators in supersy...
9,8,220,8_epidemics_epidemic_bayesian_stochastic,"[epidemics, epidemic, bayesian, stochastic, in...",[Bayesian inference for stochastic epidemic mo...


In [72]:
topics_over_time_mathematics = topic_model_mathematics.topics_over_time(
    docs=mathematics_df["Text"],
    timestamps=mathematics_df["Date"]
)

In [73]:
topic_model_mathematics.visualize_topics_over_time(topics_over_time_mathematics, top_n_topics=10)